## Анализ текста с помощью готовой модели

**Ссылка на модель:** [тык](https://huggingface.co/bhadresh-savani/distilbert-base-uncased-emotion)

In [1]:
import pandas as pd 
from transformers import pipeline

In [2]:
model = pipeline("text-classification", 
                 model = 'bhadresh-savani/distilbert-base-uncased-emotion', 
                 return_all_scores = True,
                 device = -1) #cpu

Device set to use cpu
/home/faust/.local/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [6]:
df = pd.read_json("data/interpol.json")

In [4]:
def analyze_song(song):
    all_scores = []
    if song is None:
        return pd.Series() #пустой результат

    for section in song:
        try:
            curr_section = ' '.join(section)
            prediction = model(curr_section)[0]  #первый элемент, иначе будет ещё один вложенный список на выходе
            prediction = {i['label'] : i['score'] for i in prediction}

            all_scores.append(prediction)
            print(f"Проанализирована cекция {section[0]}")
        except Exception as e:
            print(f"Ошибка  в {section[0]} : {e}")

    #усредняем результат всех строк для каждой песни
    df_emotions = pd.DataFrame(all_scores).mean()

    return df_emotions


In [5]:
df_emotions = df['Lyrics Cleaned'].apply(analyze_song)
df = pd.concat([df, df_emotions], axis = 1)
df.head()


Проанализирована cекция Surprise, sometimes, will come around
Проанализирована cекция I wish I could eat the salt off of your lost faded lips
Проанализирована cекция But she can read, she can read, she can read, she can read
Проанализирована cекция But it's different now that I'm poor and aging
Проанализирована cекция And we can find new ways of living
Проанализирована cекция But she can read, she can read, she can read, she can read
Проанализирована cекция It's different now that I'm poor and aging
Проанализирована cекция It's in the way that she posed
Проанализирована cекция It's in the way that she walks
Проанализирована cекция I had seven faces
Проанализирована cекция But New York cares (Got to be some more change in my life)
Проанализирована cекция Subway, she is a porno
Проанализирована cекция It's up to me now, turn on the bright lights
Проанализирована cекция New York cares (Got to be some more change in my life)
Проанализирована cекция It's up to me now, turn on the bright lig

,Artist,Album Name,Track Name,Key,Tempo,Mode,Lyrics Cleaned,sadness,joy,love,anger,fear,surprise
0,Interpol,Turn On the Bright Lights,Untitled,Ab,92,Major,"[[Surprise, sometimes, will come around, Surpr...",0.004170,0.457357,0.002486,0.008452,0.022101,0.505434
1,Interpol,Turn On the Bright Lights,Obstacle 1,F,121,Major,[[I wish I could eat the salt off of your lost...,0.628548,0.258516,0.002606,0.106197,0.003179,0.000955
2,Interpol,Turn On the Bright Lights,NYC,F,149,Major,"[[I had seven faces, Thought I knew which one ...",0.024825,0.498694,0.012795,0.240601,0.008274,0.214810
3,Interpol,Turn On the Bright Lights,PDA,F#,138,Minor,"[[Yours is the only version of my desertion, T...",0.178159,0.325133,0.006691,0.430700,0.057378,0.001939
4,Interpol,Turn On the Bright Lights,Say Hello to the Angels,A,103,Minor,"[[I want your silent parts, the parts the bird...",0.212803,0.400630,0.112457,0.251379,0.020475,0.002257


In [6]:
gloom_df = df.sort_values(by="sadness", ascending=False)

gloom_df

,Artist,Album Name,Track Name,Key,Tempo,Mode,Lyrics Cleaned,sadness,joy,love,anger,fear,surprise
1,Interpol,Turn On the Bright Lights,Obstacle 1,F,121,Major,[[I wish I could eat the salt off of your lost...,0.628548,0.258516,0.002606,0.106197,0.003179,0.000955
67,Interpol,The Other Side Of Make-Believe,Into the Night,C,136,Major,"[[Step around, don't stand there waiting, On a...",0.536693,0.216694,0.031524,0.175091,0.039094,0.000904
10,Interpol,Turn On the Bright Lights,Leif Erikson,G,109,Major,"[[She says it helps with the lights out, Her r...",0.483146,0.187726,0.147521,0.176763,0.004355,0.000489
46,Interpol,El Pintor,My Blue Supreme,C,103,Major,"[[Fade out, You've already made a scene around...",0.444619,0.410290,0.004952,0.134042,0.005180,0.000916
55,Interpol,Marauder,Flight of Fancy,C,133,Major,"[[Who reigns in that silence, When you sleep i...",0.417772,0.473934,0.004231,0.090228,0.001896,0.011940
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,Interpol,Antics,Length of Love,Bb,127,Major,"[[This could be destiny, Oh, sweetheart, I've ...",0.008505,0.712618,0.023246,0.252549,0.002125,0.000957
16,Interpol,Antics,Not Even Jail,C,125,Minor,"[[I'll lay down my glasses, I'll lay down in h...",0.007692,0.334249,0.211688,0.112321,0.333095,0.000954
8,Interpol,Turn On the Bright Lights,Roland,E,146,Minor,"[[Shhhh, C'mon, c'mon, Hey], [My best friend's...",0.006880,0.164443,0.259693,0.291485,0.275603,0.001896
0,Interpol,Turn On the Bright Lights,Untitled,Ab,92,Major,"[[Surprise, sometimes, will come around, Surpr...",0.004170,0.457357,0.002486,0.008452,0.022101,0.505434


In [7]:
df.to_json('data/interpol_analyzed_lyrics.json')

## Тональность + Темп
**Важно:** при выборе примерного числового значения для каждого сочетания я ориентировалась на информацию [отсюда](https://legacy.wmich.edu/mus-theo/courses/keys.html), чтобы получить общие представления об аккордах. Выводы очень субъективные.

In [7]:
df = pd.read_json('data/interpol_analyzed_lyrics.json')
df['Mode'] = df['Mode'].apply(str.strip)
df['Key'] = df['Key'].apply(str.strip)
print(df.value_counts(['Key', 'Mode']))

Key  Mode 
G    Major    11
C    Major    10
B    Minor     8
E    Minor     7
A    Minor     7
D    Major     5
     Minor     5
F#   Minor     4
F    Major     4
Ab   Major     3
Bb   Major     3
A    Major     2
C    Minor     2
E    Major     1
F    Minor     1
B    Major     1
Ab   Minor     1
G    Minor     1
Name: count, dtype: int64


In [9]:
#пропущенные значения это D минор, всё, вроде, нормально? По количеству совпадает
#типо, при выборе key == None он НИЧЕГО НЕ НАХОДИТ
missing_key = df[(df['Key'] == 'D') & (df['Mode']== 'Minor')]
print(missing_key[['Track Name', 'Album Name']])

               Track Name          Album Name
18                 C'mere              Antics
20  A Time to Be So Small              Antics
26      Pace Is the Trick  Our Love to Admire
31         The Lighthouse  Our Love to Admire
64    It Probably Matters            Marauder


In [3]:
key_values = {
    'Bb Minor': {'sadness': 0.4, 'joy': -0.5, 'anger': 0.2, 'fear': 0.3},
    'Ab Minor': {'sadness': 0.38, 'joy': -0.48, 'anger': 0.15, 'fear': 0.25},
    'D Minor': {'sadness': 0.35, 'joy': -0.5, 'love': 0.1, 'anger': 0.15},
    'F Minor': {'sadness': 0.35, 'joy': -0.45},
    'G Minor': {'sadness': 0.3, 'joy': -0.45, 'anger': 0.2, 'fear': 0.2},
    'C Minor': {'sadness': 0.3, 'joy': -0.45, 'love': 0.2},
    'B Minor': {'sadness': 0.4, 'joy': -0.35},
    'F# Minor': {'sadness': 0.35, 'joy': -0.4, 'anger': 0.1, 'fear': 0.1},
    'A Minor': {'sadness': 0.2, 'joy': -0.4, 'love': 0.2, 'fear': 0.1},
    'E Minor': {'sadness': 0.25, 'joy': -0.4, 'love': 0.3},
    'G# Minor': {'sadness': 0.25, 'joy': -0.3, 'fear': 0.1},

    'Ab Major': {'sadness': 0.05, 'joy': 0.1, 'love': 0.15},
    'Bb Major': {'sadness': 0.0, 'joy': 0.1, 'love': 0.05},
    'C Major': {'sadness': -0.1, 'joy': 0.1},
    'A Major': {'sadness': 0.1, 'joy': 0.1, 'love': 0.1},
    'F Major': {'sadness': -0.1, 'joy': 0.15, 'love': 0.1},
    'G Major': {'sadness': 0.1, 'joy': 0.15, 'love': 0.1},
    'D Major': {'sadness': -0.05, 'joy': 0.2, 'love': 0.15},
    'E Major': {'sadness': -0.05, 'joy': 0.25, 'love': 0.2},
    'B Major': {'sadness': -0.1, 'joy': 0.3, 'love': 0.1},
}

emotions = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [4]:
def apply_key_emotions(song, emotions = emotions, key_values = key_values):
    key_and_mode = f"{song["Key"]} {song["Mode"]}"

    if key_and_mode in key_values:
        emotions_adjusted = song[emotions].copy() #копия, чтобы не менять пока исходный дф

        #применение весов
        for emotion, weight in key_values[key_and_mode].items():
            if emotion in emotions_adjusted.index:
                emotions_adjusted[emotion] += weight
        return emotions_adjusted
    
    else:
        return song[emotions]


In [5]:
def apply_bpm_emotions(song):
    bpm = song['Tempo']
    emotions_adjusted = song[emotions].copy() #копия, чтобы не менять пока исходный дф
    
    effects = {'sadness': 0, 'joy': 0,
               'anger': 0, 'fear': 0}
    
    if bpm <= 65:
        effects.update({'sadness': 0.4, 'fear': 0.3, 'joy': -0.3})
    elif 66 <= bpm <= 80:
        effects.update({'sadness': 0.3, 'fear': 0.2, 'joy' : -0.25})
    elif 81 <= bpm <= 100:
        effects.update({'sadness': 0.2, 'anger': 0.1, 'joy' : -0.15})
    elif 101 <= bpm <= 120:
        effects.update({'sadness': 0.1, 'joy': -0.1, 'anger': 0.15})
    elif 121 <= bpm <= 140:
        effects.update({'anger': 0.2, 'fear': 0.1, 'joy': -0.05, 'sadness' : 0.1})
    elif bpm >= 141:
        effects.update({'anger': 0.25, 'fear': 0.25, 'joy': -0.1})
    
    for emotion, weight in effects.items():
        if emotion in emotions_adjusted:
            emotions_adjusted[emotion] += weight
            
    return emotions_adjusted

In [10]:
def normalize_emotions (song):
    emotions_values = song[emotions].copy()
    
    total = emotions_values.sum()

    if total > 0:
        normalized = emotions_values / total
    else:
        #если cумма 0, распределяем поровну
        normalized = pd.Series([1/len(emotions)] * len(emotions), index=emotions)
    return normalized

In [6]:
def apply_all(song):
    result = song[emotions].copy()
    
    #копия песни с новыми значениями
    temp_song = song.copy()
    temp_song[emotions] = result
    
    result = apply_key_emotions(temp_song)
    temp_song[emotions] = result
    
    result = apply_bpm_emotions(temp_song)
    temp_song[emotions] = result

    result = normalize_emotions(temp_song)
    return result


In [16]:
df = pd.read_json('data/interpol_analyzed_lyrics.json')

df[emotions] = df.apply(apply_all, axis = 1)

df.sort_values(by='sadness', ascending=False)


,Artist,Album Name,Track Name,Key,Tempo,Mode,Lyrics Cleaned,sadness,joy,love,anger,fear,surprise
7,Interpol,Turn On the Bright Lights,Stella Was a Diver and She Was Always Down,F,114,Minor,"[[This one's called ""Stella Was a Diver and Sh...",0.667846,-0.440322,0.170793,0.321842,0.277053,0.002788
40,Interpol,Interpol,All of the Ways,B,79,Minor,"[[Tell me you're mine, Baby, tell me you're mi...",0.563211,-0.038509,0.005016,0.303941,0.164985,0.001357
1,Interpol,Turn On the Bright Lights,Obstacle 1,F,121,Major,[[I wish I could eat the salt off of your lost...,0.539665,0.154456,0.001931,0.226813,0.076429,0.000707
69,Interpol,The Other Side Of Make-Believe,Something Changed,C,120,Minor,"[[I waded through shame for this], [I waded th...",0.531581,-0.064308,0.169813,0.355747,0.006169,0.000998
20,Interpol,Antics,A Time to Be So Small,D,87,Minor,"[[We saw you from the urchin's side, from unde...",0.526166,-0.339854,0.081830,0.599692,0.130073,0.002093
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,Interpol,Interpol,Try It On,C,132,Major,"[[I need the sunlight, to keep me above, These...",0.024219,0.370365,0.013295,0.503679,0.087089,0.001353
75,Interpol,The Other Side Of Make-Believe,Go Easy (Palermo),C,108,Major,"[[Go easy, Go easy, Go easy, Go easy, Don't ma...",0.019548,0.480513,0.004612,0.485515,0.008600,0.001212
58,Interpol,Marauder,Mountain Child,C,129,Major,"[[Mountain child, You're up in the trees like ...",0.006703,0.168732,0.001223,0.601164,0.153710,0.068468
51,Interpol,El Pintor,Twice as Hard,D,165,Major,"[[There were days of no surprise on the wing, ...",-0.006724,0.214364,0.093097,0.548175,0.150379,0.000708


## Обложки альбомов

In [19]:
album_covers = {"Turn On the Bright Lights" : "https://lastfm.freetls.fastly.net/i/u/3fe8c6984be61796f337afd7d58e921a",
                "Antics" : "https://lastfm.freetls.fastly.net/i/u/dbedf82b014c45faad56d5a34609091c",
                "Our Love to Admire" : "https://lastfm.freetls.fastly.net/i/u/15d2aa8a66d448358eef9e933a8c0a17",
                "Interpol" : "https://lastfm.freetls.fastly.net/i/u/17e545b4a2af43dc80e6b91f87f2827f",
                "El Pintor" : "https://lastfm.freetls.fastly.net/i/u/65d6575f771343c6c3a4139efb1bfdf3", 
                "Marauder" : "https://lastfm.freetls.fastly.net/i/u/49bf393f7cb6f8f86ca4fe6f797d5edf",
                "The Other Side Of Make-Believe" : "https://lastfm.freetls.fastly.net/i/u/32624d570e74625844267de63586a990"
                }

df['Album Cover'] = df['Album Name'].map(album_covers)
df.head()


,Artist,Album Name,Track Name,Key,Tempo,Mode,Lyrics Cleaned,sadness,joy,love,anger,fear,surprise,Album Cover
0,Interpol,Turn On the Bright Lights,Untitled,Ab,92,Major,"[[Surprise, sometimes, will come around, Surpr...",0.175290,0.280936,0.105163,0.074794,0.015242,0.348575,https://lastfm.freetls.fastly.net/i/u/3fe8c698...
1,Interpol,Turn On the Bright Lights,Obstacle 1,F,121,Major,[[I wish I could eat the salt off of your lost...,0.539665,0.154456,0.001931,0.226813,0.076429,0.000707,https://lastfm.freetls.fastly.net/i/u/3fe8c698...
2,Interpol,Turn On the Bright Lights,NYC,F,149,Major,"[[I had seven faces, Thought I knew which one ...",-0.048500,0.353996,0.072771,0.316517,0.166629,0.138587,https://lastfm.freetls.fastly.net/i/u/3fe8c698...
3,Interpol,Turn On the Bright Lights,PDA,F#,138,Minor,"[[Yours is the only version of my desertion, T...",0.418772,-0.083245,0.004461,0.487134,0.171585,0.001293,https://lastfm.freetls.fastly.net/i/u/3fe8c698...
4,Interpol,Turn On the Bright Lights,Say Hello to the Angels,A,103,Minor,"[[I want your silent parts, the parts the bird...",0.410242,-0.079496,0.249965,0.321103,0.096380,0.001806,https://lastfm.freetls.fastly.net/i/u/3fe8c698...


## Мрачность (Gloom Score)

In [53]:
def gloom_score(song):
    weights = {
        'sadness': 0.7,
        'joy': -0.25,
        'anger': 0.2,
        'fear': 0.25,
        'surprise': 0.15,
        'love': -0.3}
    
    weighted_sum = sum(song[emotion] * weight for emotion, weight in weights.items())
    
    total_weight = sum(abs(weight) for weight in weights.values())
    
    normalized_score = (weighted_sum + total_weight) / (2 * total_weight)
    
    #0 или 1, если есть выбросы
    return max(0, min(1, normalized_score))

In [54]:
df['Gloom'] = df.apply(gloom_score, axis = 1)
df.sort_values(by ='Gloom', ascending=False)

,Artist,Album Name,Track Name,Key,Tempo,Mode,Lyrics Cleaned,sadness,joy,love,anger,fear,surprise,Album Cover,Gloom
7,Interpol,Turn On the Bright Lights,Stella Was a Diver and She Was Always Down,F,114,Minor,"[[This one's called ""Stella Was a Diver and Sh...",0.667846,-0.440322,0.170793,0.321842,0.277053,0.002788,https://lastfm.freetls.fastly.net/i/u/3fe8c698...,0.678482
68,Interpol,The Other Side Of Make-Believe,Mr.Credit,F#,74,Minor,"[[There's nothing left to stop it, How long we...",0.503453,-0.380557,0.007601,0.566524,0.301787,0.001193,https://lastfm.freetls.fastly.net/i/u/32624d57...,0.671407
20,Interpol,Antics,A Time to Be So Small,D,87,Minor,"[[We saw you from the urchin's side, from unde...",0.526166,-0.339854,0.081830,0.599692,0.130073,0.002093,https://lastfm.freetls.fastly.net/i/u/dbedf82b...,0.657163
9,Interpol,Turn On the Bright Lights,The New,G,113,Minor,"[[I wish I could live free, I hope it's not be...",0.443321,-0.175208,0.003281,0.582278,0.145778,0.000551,https://lastfm.freetls.fastly.net/i/u/3fe8c698...,0.636791
40,Interpol,Interpol,All of the Ways,B,79,Minor,"[[Tell me you're mine, Baby, tell me you're mi...",0.563211,-0.038509,0.005016,0.303941,0.164985,0.001357,https://lastfm.freetls.fastly.net/i/u/17e545b4...,0.636381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,Interpol,Antics,Length of Love,Bb,127,Major,"[[This could be destiny, Oh, sweetheart, I've ...",0.072337,0.508412,0.048831,0.301700,0.068083,0.000638,https://lastfm.freetls.fastly.net/i/u/dbedf82b...,0.496308
2,Interpol,Turn On the Bright Lights,NYC,F,149,Major,"[[I had seven faces, Thought I knew which one ...",-0.048500,0.353996,0.072771,0.316517,0.166629,0.138587,https://lastfm.freetls.fastly.net/i/u/3fe8c698...,0.494991
72,Interpol,The Other Side Of Make-Believe,Greenwich,G,141,Major,"[[Kept along, I got nothing but my own dreams,...",0.058704,0.510472,0.057717,0.228783,0.144100,0.000224,https://lastfm.freetls.fastly.net/i/u/32624d57...,0.494047
14,Interpol,Antics,Take You on a Cruise,D,118,Major,"[[I'm timeless is like a broken watch, I make ...",0.080143,0.486066,0.113381,0.312189,0.007725,0.000495,https://lastfm.freetls.fastly.net/i/u/dbedf82b...,0.490544


In [55]:
df.to_json('data/interpol_complete.json')